In [1]:
import os
import torch
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
import plotly.graph_objects as go
from torch.utils.data import DataLoader

def minmax_scale(tensor):
    tensor_min = tensor.min()
    tensor_max = tensor.max()
    scaled_tensor = (tensor - tensor_min) / (tensor_max - tensor_min)
    return scaled_tensor, tensor_min, tensor_max

def inverse_minmax_scale(scaled_tensor, tensor_min, tensor_max):
    original_tensor = scaled_tensor * (tensor_max - tensor_min) + tensor_min
    return original_tensor
    
def smape(y_true, y_pred):
    return 100 * torch.mean(2 * torch.abs(y_true - y_pred) / (torch.abs(y_true) + torch.abs(y_pred))).item()

def rmse(y_true, y_pred):
    return torch.sqrt(torch.mean((y_true - y_pred) ** 2))
    
def mae(y_true, y_pred):
    return torch.mean(torch.abs(y_true - y_pred))


window_size = 24
horizon = 24
train_prop = 0.6
val_prop = 0.2

df1 = pd.read_csv('detached1.csv')
df2 = pd.read_csv('detached2.csv')
df1 = pd.read_csv('detached1.csv').drop(columns=['Datetime'])
df2 = pd.read_csv('detached2.csv').drop(columns=['Datetime'])

class SlidingWindowsDataset(torch.utils.data.Dataset):
    def __init__(self, data, window_size=24, horizon=24):
        self.data = data
        self.window_size = window_size
        self.horizon = horizon
        
        self.feature_mins = self.data.min(axis=0).values
        self.feature_maxs = self.data.max(axis=0).values
    
    def __len__(self):
        return len(self.data) - (self.window_size + self.horizon) + 1

    def __getitem__(self, idx):
        window = self.data[idx: idx + self.window_size + self.horizon]
        scaled_past_all_features = window[:self.window_size]
        future_energy = window[self.window_size:, -1:]
        
        #scaled_past_all_features = (past_all_features - self.feature_mins) / (self.feature_maxs - self.feature_mins)
        
        scaled_future_energy, energy_min, energy_max = minmax_scale(future_energy)

        return scaled_past_all_features, scaled_future_energy, (energy_min, energy_max)

import torch

df_dict = {
    'Detached_House_1': df1,
    'Detached_House_2':df2
}

train_datasets = {}
val_datasets = {}
test_datasets = {}

for dataset_name, dataset_df in df_dict.items():
    # Convert to tensor and perform the split
    data_tensor = torch.from_numpy(dataset_df.values).float()

    data_shape = data_tensor.shape[0]
    train_end = int(data_shape * train_prop)
    val_end = int(data_shape * (train_prop + val_prop))

    train_data = data_tensor[:train_end]
    val_data = data_tensor[train_end:val_end]
    test_data = data_tensor[val_end:]
    
    train_datasets[dataset_name] = SlidingWindowsDataset(train_data, window_size, horizon)
    val_datasets[dataset_name] = SlidingWindowsDataset(val_data, window_size, horizon)
    test_datasets[dataset_name] = SlidingWindowsDataset(test_data, window_size, horizon)

for dwelling_type, dataset in train_datasets.items():
    item = dataset[0]  # This gets the first item in the dataset
    past_features, future_energy, (energy_min, energy_max) = item
    
    print(f"Data for {dwelling_type}:")
    print("Past Features:")
    print(past_features)
    print("Future Energy:")
    print(future_energy)
    print("Energy Min:", energy_min.item())
    print("Energy Max:", energy_max.item())
    print("\n" + "="*50 + "\n")

Data for Detached_House_1:
Past Features:
tensor([[2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00, 0.0000e+00, 1.8000e+03],
        [2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00, 1.0000e+00, 1.9600e+03],
        [2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00, 2.0000e+00, 1.8000e+03],
        [2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00, 3.0000e+00, 1.9000e+03],
        [2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00, 4.0000e+00, 1.7900e+03],
        [2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00, 5.0000e+00, 1.8300e+03],
        [2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00, 6.0000e+00, 1.8400e+03],
        [2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00, 7.0000e+00, 1.9800e+03],
        [2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00, 8.0000e+00, 2.1100e+03],
        [2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00, 9.0000e+00, 2.1400e+03],
        [2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00, 1.0000e+01, 2.0200e+03],
        [2.0000e+00, 6.0000e+01, 1.0000e+00, 0.0000e+00,

In [4]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, dropout=0.8)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

if device.type == 'cuda':
    print("Training will be performed on GPU")
else:
    print("Training will be performed on CPU")

learning_rate = 0.001

def train(model, data_loader, optimizer, loss_fn):
    model.train()
    losses = []
    metrics = {'MAE': [], 'RMSE': [], 'SMAPE': []}

    for inputs, scaled_targets, future_energy_min_max in data_loader:
        inputs, scaled_targets = inputs.to(device), scaled_targets.to(device)
        #print("Scaled Targets:", scaled_targets[0].cpu().numpy())  # Print the first scaled target
        targets_min, targets_max = future_energy_min_max
        targets_min, targets_max = targets_min.to(device), targets_max.to(device)
        #print("Min for targets:", targets_min[0].item())   # Print min value used for scaling for the first target
        #print("Max for targets:", targets_max[0].item())   # Print max value used for scaling for the first target

        outputs = model(inputs)
        #print(outputs.shape)
        loss = loss_fn(outputs, scaled_targets)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        optimizer.step()

        targets = inverse_minmax_scale(scaled_targets, targets_min, targets_max)
        #print("Inverse Scaled Targets:", targets[0].cpu().numpy())  # Print the first inverse scaled target
        outputs = inverse_minmax_scale(outputs, targets_min, targets_max)
        #print("Inverse Scaled Outputs:", outputs[0].detach().cpu().numpy())    # Print the first inverse scaled output

        metrics['MAE'].append(mae(outputs, targets).item())
        metrics['RMSE'].append(rmse(outputs, targets).item())
        metrics['SMAPE'].append(smape(outputs, targets))

    return np.mean(losses), {k: np.mean(v) for k, v in metrics.items()}

def evaluate(model, data_loader, loss_fn):
    model.eval()
    losses = []
    metrics = {'MAE': [], 'RMSE': [], 'SMAPE': []}

    with torch.no_grad():
        for inputs, scaled_targets, future_energy_min_max in data_loader:
            inputs, scaled_targets = inputs.to(device), scaled_targets.to(device)
            targets_min, targets_max = future_energy_min_max
            targets_min, targets_max = targets_min.to(device), targets_max.to(device)

            outputs = model(inputs)
            loss = loss_fn(outputs, scaled_targets)
            losses.append(loss.item())

            targets = inverse_minmax_scale(scaled_targets, targets_min, targets_max)
            outputs = inverse_minmax_scale(outputs, targets_min, targets_max)

            metrics['MAE'].append(mae(outputs, targets).item())
            metrics['RMSE'].append(rmse(outputs, targets).item())
            metrics['SMAPE'].append(smape(outputs, targets))

    return np.mean(losses), {k: np.mean(v) for k, v in metrics.items()}
    
model = LSTM(input_size=6, hidden_size=32, output_size=1).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-7)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5, verbose=True)
loss_fn = nn.L1Loss()

def train_and_validate(datasets, num_epochs, patience, loss_fn):
    plots_dir = "./LSTM_MSE/"
    
    for dataset_name in datasets['train'].keys():
        print(f"Training for dataset: {dataset_name}")
        
        train_loader = DataLoader(datasets['train'][dataset_name], batch_size=1, shuffle=False)
        val_loader = DataLoader(datasets['val'][dataset_name], batch_size=1, shuffle=False)
        
        best_val_loss = np.inf
        epochs_no_improve = 0

        train_loss_history = []
        val_loss_history = []
        train_mae_history = []
        val_mae_history = []
        train_rmse_history = []
        val_rmse_history = []
        train_smape_history = []
        val_smape_history = []

        for epoch in range(num_epochs):
            train_loss, train_metrics = train(model, train_loader, optimizer, loss_fn)
            val_loss, val_metrics = evaluate(model, val_loader, loss_fn)
            scheduler.step(val_loss)
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

            if epochs_no_improve == patience:
                print(f"Early stopping triggered after {epoch + 1} epochs for dataset {dataset_name}.")
                break

            train_loss_history.append(train_loss)
            val_loss_history.append(val_loss)
            train_mae_history.append(train_metrics['MAE'])
            val_mae_history.append(val_metrics['MAE'])
            train_rmse_history.append(train_metrics['RMSE'])
            val_rmse_history.append(val_metrics['RMSE'])
            train_smape_history.append(train_metrics['SMAPE'])
            val_smape_history.append(val_metrics['SMAPE'])

            print(f"Epoch {epoch + 1}/{num_epochs} for dataset {dataset_name}")
            print(f"Train Loss: {train_loss:.2f}, Validation Loss: {val_loss:.2f}")
            print(f"Train MAE: {train_metrics['MAE']:.2f}, Validation MAE: {val_metrics['MAE']:.2f}")
            print(f"Train RMSE: {train_metrics['RMSE']:.2f}, Validation RMSE: {val_metrics['RMSE']:.2f}")
            print(f"Train SMAPE: {train_metrics['SMAPE']:.2f}, Validation SMAPE: {val_metrics['SMAPE']:.2f}")

        # Create plots and save them for each dataset
        def plot_metrics(history1, history2, title, ylabel, filename):
            fig = go.Figure()
            fig.add_trace(go.Scatter(y=history1, mode='lines', name='Train'))
            fig.add_trace(go.Scatter(y=history2, mode='lines', name='Validation'))
            fig.update_layout(title=title, xaxis_title='Epoch', yaxis_title=ylabel)
            fig.write_image(f"{plots_dir}/{dataset_name}_{filename}")

        plot_metrics(train_loss_history, val_loss_history, 'Loss vs Epoch', 'Loss', 'drop_loss_plot.png')
        plot_metrics(train_mae_history, val_mae_history, 'MAE vs Epoch', 'MAE', 'drop_mae_plot.png')
        plot_metrics(train_rmse_history, val_rmse_history, 'RMSE vs Epoch', 'RMSE', 'drop_rmse_plot.png')
        plot_metrics(train_smape_history, val_smape_history, 'SMAPE vs Epoch', 'SMAPE', 'drop_smape_plot.png')

        # Save the model for each dataset
        torch.save(model.state_dict(), f'./LSTM_MSE/LSTM_{loss_fn}_{dataset_name}.pth')

datasets = {
    'train': train_datasets,
    'val': val_datasets,
    'test': test_datasets
}



patience = 10
num_epochs = 500

train_and_validate(datasets, num_epochs, patience, loss_fn)

Training will be performed on GPU
Training for dataset: Detached_House_1
Epoch 1/500 for dataset Detached_House_1
Train Loss: 0.30, Validation Loss: 0.31
Train MAE: 381.82, Validation MAE: 686.80
Train RMSE: 468.44, Validation RMSE: 792.35
Train SMAPE: 24.57, Validation SMAPE: 29.73
Epoch 2/500 for dataset Detached_House_1
Train Loss: 0.24, Validation Loss: 0.30
Train MAE: 308.98, Validation MAE: 684.20
Train RMSE: 381.15, Validation RMSE: 785.03
Train SMAPE: 19.26, Validation SMAPE: 29.64
Epoch 3/500 for dataset Detached_House_1
Train Loss: 0.24, Validation Loss: 0.30
Train MAE: 307.68, Validation MAE: 683.93
Train RMSE: 378.96, Validation RMSE: 784.55
Train SMAPE: 19.20, Validation SMAPE: 29.63
Epoch 4/500 for dataset Detached_House_1
Train Loss: 0.24, Validation Loss: 0.30
Train MAE: 307.68, Validation MAE: 683.89
Train RMSE: 378.94, Validation RMSE: 784.53
Train SMAPE: 19.20, Validation SMAPE: 29.63
Epoch 5/500 for dataset Detached_House_1
Train Loss: 0.24, Validation Loss: 0.30
Tr

In [7]:
class GRU(nn.Module):
    def __init__(self, num_features):
        super(GRU, self).__init__()
        self.gru = nn.GRU(input_size=num_features, hidden_size=64, batch_first=True,dropout=0.1)
        self.fc = nn.Linear(64, 1)

    def forward(self, x):
        x, _ = self.gru(x)
        output = self.fc(x)
        return output


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if device.type == 'cuda':
    print("Training will be performed on GPU")
else:
    print("Training will be performed on CPU")

learning_rate = 0.001

def train(model, data_loader, optimizer, loss_fn):
    model.train()
    losses = []
    metrics = {'MAE': [], 'RMSE': [], 'SMAPE': []}

    for inputs, scaled_targets, future_energy_min_max in data_loader:
        inputs, scaled_targets = inputs.to(device), scaled_targets.to(device)
        #print("Scaled Targets:", scaled_targets[0].cpu().numpy())  # Print the first scaled target
        targets_min, targets_max = future_energy_min_max
        targets_min, targets_max = targets_min.to(device), targets_max.to(device)
        #print("Min for targets:", targets_min[0].item())   # Print min value used for scaling for the first target
        #print("Max for targets:", targets_max[0].item())   # Print max value used for scaling for the first target

        outputs = model(inputs)
        #print("Outputs:", outputs[0].detach().cpu().numpy())
        loss = loss_fn(outputs, scaled_targets)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        targets = inverse_minmax_scale(scaled_targets, targets_min, targets_max)
        #print("Inverse Scaled Targets:", targets[0].cpu().numpy())  # Print the first inverse scaled target
        outputs = inverse_minmax_scale(outputs, targets_min, targets_max)
        #print("Inverse Scaled Outputs:", outputs[0].detach().cpu().numpy())    # Print the first inverse scaled output

        metrics['MAE'].append(mae(outputs, targets).item())
        metrics['RMSE'].append(rmse(outputs, targets).item())
        metrics['SMAPE'].append(smape(outputs, targets))

    return np.mean(losses), {k: np.mean(v) for k, v in metrics.items()}

def evaluate(model, data_loader, loss_fn):
    model.eval()
    losses = []
    metrics = {'MAE': [], 'RMSE': [], 'SMAPE': []}

    with torch.no_grad():
        for inputs, scaled_targets, future_energy_min_max in data_loader:
            inputs, scaled_targets = inputs.to(device), scaled_targets.to(device)
            targets_min, targets_max = future_energy_min_max
            targets_min, targets_max = targets_min.to(device), targets_max.to(device)

            outputs = model(inputs)
            loss = loss_fn(outputs, scaled_targets)
            losses.append(loss.item())

            targets = inverse_minmax_scale(scaled_targets, targets_min, targets_max)
            outputs = inverse_minmax_scale(outputs, targets_min, targets_max)

            metrics['MAE'].append(mae(outputs, targets).item())
            metrics['RMSE'].append(rmse(outputs, targets).item())
            metrics['SMAPE'].append(smape(outputs, targets))

    return np.mean(losses), {k: np.mean(v) for k, v in metrics.items()}
    
model = GRU(num_features=6).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-7)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5, verbose=True)
loss_fn = nn.L1Loss()

def train_and_validate(datasets, num_epochs, patience, loss_fn):
    plots_dir = "./GRU_MSE/"
    
    for dataset_name in datasets['train'].keys():
        print(f"Training for dataset: {dataset_name}")
        
        train_loader = DataLoader(datasets['train'][dataset_name], batch_size=1, shuffle=False)
        val_loader = DataLoader(datasets['val'][dataset_name], batch_size=1, shuffle=False)
        
        best_val_loss = np.inf
        epochs_no_improve = 0

        train_loss_history = []
        val_loss_history = []
        train_mae_history = []
        val_mae_history = []
        train_rmse_history = []
        val_rmse_history = []
        train_smape_history = []
        val_smape_history = []

        for epoch in range(num_epochs):
            train_loss, train_metrics = train(model, train_loader, optimizer, loss_fn)
            val_loss, val_metrics = evaluate(model, val_loader, loss_fn)

            scheduler.step(val_loss)

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

            if epochs_no_improve == patience:
                print(f"Early stopping triggered after {epoch + 1} epochs for dataset {dataset_name}.")
                break

            train_loss_history.append(train_loss)
            val_loss_history.append(val_loss)
            train_mae_history.append(train_metrics['MAE'])
            val_mae_history.append(val_metrics['MAE'])
            train_rmse_history.append(train_metrics['RMSE'])
            val_rmse_history.append(val_metrics['RMSE'])
            train_smape_history.append(train_metrics['SMAPE'])
            val_smape_history.append(val_metrics['SMAPE'])

            print(f"Epoch {epoch + 1}/{num_epochs} for dataset {dataset_name}")
            print(f"Train Loss: {train_loss:.2f}, Validation Loss: {val_loss:.2f}")
            print(f"Train MAE: {train_metrics['MAE']:.2f}, Validation MAE: {val_metrics['MAE']:.2f}")
            print(f"Train RMSE: {train_metrics['RMSE']:.2f}, Validation RMSE: {val_metrics['RMSE']:.2f}")
            print(f"Train SMAPE: {train_metrics['SMAPE']:.2f}, Validation SMAPE: {val_metrics['SMAPE']:.2f}")

        def plot_metrics(history1, history2, title, ylabel, filename):
            fig = go.Figure()
            fig.add_trace(go.Scatter(y=history1, mode='lines', name='Train'))
            fig.add_trace(go.Scatter(y=history2, mode='lines', name='Validation'))
            fig.update_layout(title=title, xaxis_title='Epoch', yaxis_title=ylabel)
            fig.write_image(f"{plots_dir}/{dataset_name}_{filename}")

        plot_metrics(train_loss_history, val_loss_history, 'Loss vs Epoch', 'Loss', 'drop_loss_plot.png')
        plot_metrics(train_mae_history, val_mae_history, 'MAE vs Epoch', 'MAE', 'drop_mae_plot.png')
        plot_metrics(train_rmse_history, val_rmse_history, 'RMSE vs Epoch', 'RMSE', 'drop_rmse_plot.png')
        plot_metrics(train_smape_history, val_smape_history, 'SMAPE vs Epoch', 'SMAPE', 'drop_smape_plot.png')

        torch.save(model.state_dict(), f'./GRU_MSE/GRU_{loss_fn}_{dataset_name}.pth')

datasets = {
    'train': train_datasets,
    'val': val_datasets,
    'test': test_datasets
}

patience = 10
num_epochs = 500
train_and_validate(datasets, num_epochs, patience,loss_fn)

Training will be performed on GPU
Training for dataset: Detached_House_1
Epoch 1/500 for dataset Detached_House_1
Train Loss: 0.22, Validation Loss: 0.35
Train MAE: 280.91, Validation MAE: 797.29
Train RMSE: 360.61, Validation RMSE: 993.93
Train SMAPE: 17.55, Validation SMAPE: 35.03
Epoch 2/500 for dataset Detached_House_1
Train Loss: 0.22, Validation Loss: 0.35
Train MAE: 280.03, Validation MAE: 797.45
Train RMSE: 359.21, Validation RMSE: 993.07
Train SMAPE: 17.50, Validation SMAPE: 35.06
Epoch 3/500 for dataset Detached_House_1
Train Loss: 0.22, Validation Loss: 0.35
Train MAE: 280.48, Validation MAE: 797.42
Train RMSE: 359.62, Validation RMSE: 993.03
Train SMAPE: 17.53, Validation SMAPE: 35.06
Epoch 4/500 for dataset Detached_House_1
Train Loss: 0.22, Validation Loss: 0.35
Train MAE: 280.47, Validation MAE: 797.40
Train RMSE: 359.61, Validation RMSE: 993.00
Train SMAPE: 17.53, Validation SMAPE: 35.06
Epoch 5/500 for dataset Detached_House_1
Train Loss: 0.22, Validation Loss: 0.35
Tr

In [8]:
def test(model, data_loader, loss_fn):
    model.eval()
    losses = []
    all_outputs = []
    all_targets = []
    metrics = {'MAE': [], 'RMSE': [], 'SMAPE': []}

    with torch.no_grad():
        for inputs, scaled_targets, future_energy_min_max in data_loader:
            inputs, scaled_targets = inputs.to(device), scaled_targets.to(device)
            targets_min, targets_max = future_energy_min_max
            targets_min, targets_max = targets_min.to(device), targets_max.to(device)

            outputs = model(inputs)
            loss = loss_fn(outputs, scaled_targets)
            losses.append(loss.item())

            targets = inverse_minmax_scale(scaled_targets, targets_min, targets_max)
            outputs = inverse_minmax_scale(outputs, targets_min, targets_max)

            metrics['MAE'].append(mae(outputs, targets).item())
            metrics['RMSE'].append(rmse(outputs, targets).item())
            metrics['SMAPE'].append(smape(outputs, targets))

    return np.mean(losses), {k: np.mean(v) for k, v in metrics.items()}

test_results = {}
for dataset_name in datasets['test'].keys():
    print(f"Testing for dataset: {dataset_name}")
    model = GRU(num_features=6).to(device)
    model.load_state_dict(torch.load(f'./GRU_MSE/GRU_{loss_fn}_{dataset_name}.pth'))

    test_loader = DataLoader(datasets['test'][dataset_name], batch_size=1, shuffle=False)

    test_loss, test_metrics = test(model, test_loader, loss_fn)
    
    print(f"Test Loss for {dataset_name}: {test_loss:.2f}")
    print(f"Test MAE for {dataset_name}: {test_metrics['MAE']:.2f}")
    print(f"Test RMSE for {dataset_name}: {test_metrics['RMSE']:.2f}")
    print(f"Test SMAPE for {dataset_name}: {test_metrics['SMAPE']:.2f}")
    print('-' * 50)

    test_results[dataset_name] = {
        'loss': test_loss,
        'MAE': test_metrics['MAE'],
        'RMSE': test_metrics['RMSE'],
        'SMAPE': test_metrics['SMAPE']
    }

def test_with_values(model, data_loader, loss_fn):
    model.eval()
    all_outputs = []
    all_targets = []

    with torch.no_grad():
        for inputs, scaled_targets, future_energy_min_max in data_loader:
            inputs, scaled_targets = inputs.to(device), scaled_targets.to(device)
            targets_min, targets_max = future_energy_min_max
            targets_min, targets_max = targets_min.to(device), targets_max.to(device)

            outputs = model(inputs)

            targets = inverse_minmax_scale(scaled_targets, targets_min, targets_max).cpu().numpy()
            outputs = inverse_minmax_scale(outputs, targets_min, targets_max).cpu().numpy()

            all_outputs.extend(outputs)
            all_targets.extend(targets)
        
    return all_targets, all_outputs

for dataset_name in datasets['test'].keys():
    print(f"Generating values for dataset: {dataset_name}")
    model = GRU(num_features=6).to(device)
    model.load_state_dict(torch.load(f'./GRU_MSE/GRU_{loss_fn}_{dataset_name}.pth'))

    test_loader = DataLoader(datasets['test'][dataset_name], batch_size=1, shuffle=False)

    actual_values, predicted_values = test_with_values(model, test_loader, loss_fn)
    print(len(np.ravel(actual_values)))
    print(len(np.ravel(predicted_values)))
    
    df_results = pd.DataFrame({
        'Actual': actual_values,
        'Predicted': predicted_values
    })

    df_results.to_csv(f"./GRU_MSE/{dataset_name}_results.csv", index=False)
    print(f"Saved results for {dataset_name} to {dataset_name}_results.csv")

Testing for dataset: Detached_House_1
Test Loss for Detached_House_1: 0.31
Test MAE for Detached_House_1: 645.24
Test RMSE for Detached_House_1: 717.64
Test SMAPE for Detached_House_1: 32.60
--------------------------------------------------
Testing for dataset: Detached_House_2
Test Loss for Detached_House_2: 0.20
Test MAE for Detached_House_2: 371.69
Test RMSE for Detached_House_2: 537.68
Test SMAPE for Detached_House_2: 31.47
--------------------------------------------------
Generating values for dataset: Detached_House_1
19608
19608
Saved results for Detached_House_1 to Detached_House_1_results.csv
Generating values for dataset: Detached_House_2
19608
19608
Saved results for Detached_House_2 to Detached_House_2_results.csv


In [10]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F
from torch import nn
import math
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(1, 0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

class Transformer(nn.Module):
    def __init__(self, num_features, d_model, nhead, num_layers):
        super(Transformer, self).__init__()

        self.embedding = nn.Linear(num_features, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        self.transformer = nn.Transformer(d_model, nhead, num_layers)
        self.fc = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoder(x)
        x = self.transformer.encoder(x)
        output = self.fc(x)
        return output



device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

if device.type == 'cuda':
    print("Training will be performed on GPU")
else:
    print("Training will be performed on CPU")

learning_rate = 0.0001
def train(model, data_loader, optimizer, loss_fn):
    model.train()
    losses = []
    metrics = {'MAE': [], 'RMSE': [], 'SMAPE': []}

    for inputs, scaled_targets, future_energy_min_max in data_loader:
        inputs, scaled_targets = inputs.to(device), scaled_targets.to(device)
        targets_min, targets_max = future_energy_min_max
        targets_min, targets_max = targets_min.to(device), targets_max.to(device)
        
        outputs = model(inputs)
        loss = loss_fn(outputs, scaled_targets)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

        targets = inverse_minmax_scale(scaled_targets, targets_min, targets_max)
        #print("Inverse Scaled Targets:", targets[0].cpu().numpy())  # Print the first inverse scaled target
        outputs = inverse_minmax_scale(outputs, targets_min, targets_max)
        #print("Inverse Scaled Outputs:", outputs[0].detach().cpu().numpy())    # Print the first inverse scaled output

        metrics['MAE'].append(mae(outputs, targets).item())
        metrics['RMSE'].append(rmse(outputs, targets).item())
        metrics['SMAPE'].append(smape(outputs, targets))

    return np.mean(losses), {k: np.mean(v) for k, v in metrics.items()}

def evaluate(model, data_loader, loss_fn):
    model.eval()
    losses = []
    metrics = {'MAE': [], 'RMSE': [], 'SMAPE': []}

    with torch.no_grad():
        for inputs, scaled_targets, future_energy_min_max in data_loader:
            inputs, scaled_targets = inputs.to(device), scaled_targets.to(device)
            targets_min, targets_max = future_energy_min_max
            targets_min, targets_max = targets_min.to(device), targets_max.to(device)

            outputs = model(inputs)
            loss = loss_fn(outputs, scaled_targets)
            losses.append(loss.item())

            targets = inverse_minmax_scale(scaled_targets, targets_min, targets_max)
            outputs = inverse_minmax_scale(outputs, targets_min, targets_max)

            metrics['MAE'].append(mae(outputs, targets).item())
            metrics['RMSE'].append(rmse(outputs, targets).item())
            metrics['SMAPE'].append(smape(outputs, targets))

    return np.mean(losses), {k: np.mean(v) for k, v in metrics.items()}
    

model = Transformer(num_features=6, d_model=128, nhead=4, num_layers=2).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-7)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5, verbose=True)
loss_fn = nn.L1Loss()

def train_and_validate(datasets, num_epochs, patience,loss_fn):
    plots_dir = "./Transformer_MSE/"
    
    for dataset_name in datasets['train'].keys():
        print(f"Training for dataset: {dataset_name}")
        
        train_loader = DataLoader(datasets['train'][dataset_name], batch_size=1, shuffle=False)
        val_loader = DataLoader(datasets['val'][dataset_name], batch_size=1, shuffle=False)
        
        best_val_loss = np.inf
        epochs_no_improve = 0

        train_loss_history = []
        val_loss_history = []
        train_mae_history = []
        val_mae_history = []
        train_rmse_history = []
        val_rmse_history = []
        train_smape_history = []
        val_smape_history = []

        for epoch in range(num_epochs):
            train_loss, train_metrics = train(model, train_loader, optimizer, loss_fn)
            val_loss, val_metrics = evaluate(model, val_loader, loss_fn)
            scheduler.step(val_loss)
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

            if epochs_no_improve == patience:
                print(f"Early stopping triggered after {epoch + 1} epochs for dataset {dataset_name}.")
                break

            train_loss_history.append(train_loss)
            val_loss_history.append(val_loss)
            train_mae_history.append(train_metrics['MAE'])
            val_mae_history.append(val_metrics['MAE'])
            train_rmse_history.append(train_metrics['RMSE'])
            val_rmse_history.append(val_metrics['RMSE'])
            train_smape_history.append(train_metrics['SMAPE'])
            val_smape_history.append(val_metrics['SMAPE'])

            print(f"Epoch {epoch + 1}/{num_epochs} for dataset {dataset_name}")
            print(f"Train Loss: {train_loss:.2f}, Validation Loss: {val_loss:.2f}")
            print(f"Train MAE: {train_metrics['MAE']:.2f}, Validation MAE: {val_metrics['MAE']:.2f}")
            print(f"Train RMSE: {train_metrics['RMSE']:.2f}, Validation RMSE: {val_metrics['RMSE']:.2f}")
            print(f"Train SMAPE: {train_metrics['SMAPE']:.2f}, Validation SMAPE: {val_metrics['SMAPE']:.2f}")

        def plot_metrics(history1, history2, title, ylabel, filename):
            fig = go.Figure()
            fig.add_trace(go.Scatter(y=history1, mode='lines', name='Train'))
            fig.add_trace(go.Scatter(y=history2, mode='lines', name='Validation'))
            fig.update_layout(title=title, xaxis_title='Epoch', yaxis_title=ylabel)
            fig.write_image(f"{plots_dir}/{dataset_name}_{filename}")

        plot_metrics(train_loss_history, val_loss_history, 'Loss vs Epoch', 'Loss', 'drop_loss_plot.png')
        plot_metrics(train_mae_history, val_mae_history, 'MAE vs Epoch', 'MAE', 'drop_mae_plot.png')
        plot_metrics(train_rmse_history, val_rmse_history, 'RMSE vs Epoch', 'RMSE', 'drop_rmse_plot.png')
        plot_metrics(train_smape_history, val_smape_history, 'SMAPE vs Epoch', 'SMAPE', 'drop_smape_plot.png')

        torch.save(model.state_dict(), f'./Transformer_MSE/Transformer_{loss_fn}_{dataset_name}.pth')

datasets = {
    'train': train_datasets,
    'val': val_datasets,
    'test': test_datasets
}



patience = 10
num_epochs = 500

train_and_validate(datasets, num_epochs, patience,loss_fn)

Training will be performed on GPU
Training for dataset: Detached_House_1
Epoch 1/500 for dataset Detached_House_1
Train Loss: 0.28, Validation Loss: 0.32
Train MAE: 344.49, Validation MAE: 729.57
Train RMSE: 425.51, Validation RMSE: 875.47
Train SMAPE: 22.31, Validation SMAPE: 31.66
Epoch 2/500 for dataset Detached_House_1
Train Loss: 0.26, Validation Loss: 0.32
Train MAE: 319.40, Validation MAE: 717.42
Train RMSE: 395.62, Validation RMSE: 861.89
Train SMAPE: 20.47, Validation SMAPE: 31.05
Epoch 3/500 for dataset Detached_House_1
Train Loss: 0.25, Validation Loss: 0.32
Train MAE: 310.78, Validation MAE: 718.66
Train RMSE: 386.36, Validation RMSE: 870.14
Train SMAPE: 19.78, Validation SMAPE: 31.06
Epoch 4/500 for dataset Detached_House_1
Train Loss: 0.24, Validation Loss: 0.31
Train MAE: 304.24, Validation MAE: 701.21
Train RMSE: 379.29, Validation RMSE: 841.65
Train SMAPE: 19.35, Validation SMAPE: 30.24
Epoch 5/500 for dataset Detached_House_1
Train Loss: 0.24, Validation Loss: 0.31
Tr